In [1]:
using DataFrames
using CSV

In [2]:
ENV["COLUMNS"] = 1000;

In [3]:
rawdata = DataFrame(CSV.File("../rawdata/reported_hospital_capacity_admissions_facility_level_weekly_average_timeseries_20201221_0.csv"));
data = select(rawdata,
    :hospital_name => ByRow(titlecase) => :hospital,
    :zip => :zipcode,
);
data = combine(groupby(data, :hospital), :zipcode => (x -> x[1]) => :zipcode)

,hospital,zipcode
,String,Int64?
1,Surgery Center Of Zachary,70791
2,Healthpromed,775
3,Hospital San Antonio,680
4,Cdt Susana Centeno,765
5,Centro Medico Correccional De Bayamon,960
6,Crescent City Surgical Centre,70118
7,El Paso Ltac Hospital,79902
8,Alexandria Emergency Hospital,71303
9,Hospital Industrial C.F.S.E,935


In [4]:
zipcodedata = DataFrame(CSV.File("../rawdata/zipcodes.csv"))
zipcode_lookup = Dict(row.ZIP => (lat=row.LAT, long=row.LNG) for row in eachrow(zipcodedata));

In [5]:
latlongs = [haskey(zipcode_lookup,z) ? zipcode_lookup[z] : (lat=-1,long=-1) for z in data.zipcode];
lats = [l.lat for l in latlongs];
longs = [l.long for l in latlongs];

In [6]:
count(lats .== -1) / length(lats)

0.03292094778779618

In [7]:
mean(xs) = sum(xs) / length(xs);

In [8]:
lats = [(l == -1) ? mean(lats) : l for l in lats];
longs = [(l == -1) ? mean(longs) : l for l in longs];

In [9]:
data.lat = lats;
data.long = longs;

In [10]:
data |> CSV.write("../data/hhs_locations.csv")

"../data/hhs_locations.csv"